In [51]:
### Library Imports
import pandas as pd
import yfinance as yf
import numpy as np
from scipy import stats
import datetime
import matplotlib.pyplot as plt


In [52]:
### Function to Get All Tickers
def get_tickers(url):
    
    tickers_list = []
    # Get Tickers into df
    df = pd.read_html(url)
    # Convert the df to a list
    table = df[0]  # First table contains tickers
    tickers = table['Symbol'].tolist()
    # Yahoo Finance uses '-' instead of '.' (e.g., BRK.B → BRK-B)
    tickers_list = [ticker.replace('.', '-') for ticker in tickers]

    return tickers_list

url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
full_tickers = get_tickers(url)
print(full_tickers)


['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A', 'APD', 'ABNB', 'AKAM', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AEE', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON', 'APA', 'APO', 'AAPL', 'AMAT', 'APTV', 'ACGL', 'ADM', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'AXON', 'BKR', 'BALL', 'BAC', 'BAX', 'BDX', 'BRK-B', 'BBY', 'TECH', 'BIIB', 'BLK', 'BX', 'BK', 'BA', 'BKNG', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF-B', 'BLDR', 'BG', 'BXP', 'CHRW', 'CDNS', 'CZR', 'CPT', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CAT', 'CBOE', 'CBRE', 'CDW', 'COR', 'CNC', 'CNP', 'CF', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'COIN', 'CL', 'CMCSA', 'CAG', 'COP', 'ED', 'STZ', 'CEG', 'COO', 'CPRT', 'GLW', 'CPAY', 'CTVA', 'CSGP', 'COST', 'CTRA', 'CRWD', 'CCI', 'CSX', 'CMI', 'CVS', 'DHR', 'DRI', 'DVA', '

In [53]:
### Function to Import Stock Data
def import_stock_data(tickers, start_date, end_date):
    data = pd.DataFrame()
    if len([tickers]) == 1:
        data[tickers] = yf.download(tickers, start_date, end_date)['Close']
        data = pd.DataFrame(data)
    else:
        for t in tickers:
            data[t] = yf.download(tickers, start_date, end_date)['Close']
    
    # Reset index to include the Date as a column
    data = data.reset_index()

    return data

start_date = '2025-01-01'
end_date = '2025-06-01'
tickers = ['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD']
stock_data = import_stock_data(tickers, start_date, end_date)
print(stock_data.head())


/var/folders/n5/bknqp6qs37l5l4bwv9wv3bs40000gn/T/ipykernel_77464/3755755847.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data[tickers] = yf.download(tickers, start_date, end_date)['Close']
[*********************100%***********************]  7 of 7 completed

        Date         MMM         AOS         ABT        ABBV         ACN  \
0 2025-01-02  176.135895  112.327484  345.629974  441.000000  120.629997   
1 2025-01-03  177.883118  112.713669  350.613953  430.570007  125.370003   
2 2025-01-06  176.783752  111.931412  348.116974  431.179993  129.550003   
3 2025-01-07  176.224243  112.287888  353.130737  422.630005  127.330002   
4 2025-01-08  175.213211  113.129547  354.458496  419.579987  121.839996   

        ADBE         AMD  
0  66.650047  128.434326  
1  67.907227  128.602661  
2  68.204193  129.018555  
3  67.610252  131.474380  
4  67.986420  133.217194  


In [54]:
### Compute the Percentage Change
def pct_change(tickers, df):
    # Add col with daily pct change for each ticker
    for t in tickers:
        df['PCT ' + t] = df[t].pct_change()
    
    # Drop null vals
    df.dropna(subset=['PCT ' + t for t in tickers], inplace=True)

    return df

### Function Call to Compute the Percentage Change
stock_data = pct_change(tickers, stock_data)
print(stock_data.head())


        Date         MMM         AOS         ABT        ABBV         ACN  \
1 2025-01-03  177.883118  112.713669  350.613953  430.570007  125.370003   
2 2025-01-06  176.783752  111.931412  348.116974  431.179993  129.550003   
3 2025-01-07  176.224243  112.287888  353.130737  422.630005  127.330002   
4 2025-01-08  175.213211  113.129547  354.458496  419.579987  121.839996   
5 2025-01-10  171.944534  111.208565  346.591095  405.920013  116.040001   

        ADBE         AMD   PCT MMM   PCT AOS   PCT ABT  PCT ABBV   PCT ACN  \
1  67.907227  128.602661  0.009920  0.003438  0.014420 -0.023651  0.039294   
2  68.204193  129.018555 -0.006180 -0.006940 -0.007122  0.001417  0.033341   
3  67.610252  131.474380 -0.003165  0.003185  0.014403 -0.019829 -0.017136   
4  67.986420  133.217194 -0.005737  0.007496  0.003760 -0.007217 -0.043116   
5  66.897530  129.929596 -0.018655 -0.016980 -0.022196 -0.032556 -0.047603   

   PCT ADBE   PCT AMD  
1  0.018862  0.001311  
2  0.004373  0.003234  
3 

In [55]:
### Generate Buy Signals
''' 
Generate a "BUY" signal if the stock loses more than 5% of its value in a single day
'''
def generate_signals(df, drop_threshold):
    """
    Generate 'BUY' signals if a stock drops more than the threshold in a day.
    """
    for t in tickers:
        df['Signal ' + t] = (df['PCT ' + t] < drop_threshold).astype(int)
        # Test to see the number of generated signals
        print(f"{'Signal ' + t}: {df['Signal ' + t].sum()} buy signals generated")

    return df

### Function Retun to Generate Buy Signals
stock_data = generate_signals(stock_data, drop_threshold = -0.05)
print(stock_data.tail())


Signal MMM: 3 buy signals generated
Signal AOS: 1 buy signals generated
Signal ABT: 3 buy signals generated
Signal ABBV: 1 buy signals generated
Signal ACN: 7 buy signals generated
Signal ADBE: 1 buy signals generated
Signal AMD: 2 buy signals generated
          Date         MMM         AOS         ABT        ABBV         ACN  \
97  2025-05-23  183.259995  131.300003  309.579987  407.690002  110.309998   
98  2025-05-27  185.720001  132.940002  315.429993  413.100006  114.559998   
99  2025-05-28  183.089996  132.020004  315.989990  412.230011  112.860001   
100 2025-05-29  185.619995  132.850006  317.730011  413.359985  113.029999   
101 2025-05-30  186.110001  133.580002  316.820007  415.089996  110.730003   

          ADBE         AMD   PCT MMM   PCT AOS  ...   PCT ACN  PCT ADBE  \
97   67.019997  147.619995  0.003944 -0.001521  ... -0.003613 -0.005195   
98   68.570000  149.490005  0.013424  0.012490  ...  0.038528  0.023127   
99   64.230003  148.660004 -0.014161 -0.006920  ... 

In [56]:
### Trade Simulation Function
''' 
Enter trade the day after the signal and exit after a fixed holding period (ex. 5 days)
'''
def trade_sim(ticker, df, holding_days):
    trades = []

    try:
        # Iterate over the df, stopping with the number of holding days left (avoids out of bounds error) 
        for i in range(len(df) - holding_days):
            # Identify the Buy Signals - Locate rows with a "1" indicator
            if df.iloc[i][f'Signal {ticker}'] == 1:
                # Set Entry/Exit Date - For each signal date (i), set the entry date to be the next trading day (i + 1)
                entry_idx = i + 1
                exit_idx = i + holding_days

                entry_date = df.index[entry_idx]
                exit_date = df.index[exit_idx]
                # Record Entry and Exit Prices
                entry_price = df.iloc[entry_idx][ticker]
                exit_price = df.iloc[exit_idx][ticker]
                # Compute the Returns for the Asset
                rets = (exit_price / entry_price) - 1

                trades.append({
                    'Ticker': ticker,
                    'Entry Date': entry_date,
                    'Exit Date': exit_date,
                    'Entry Price': entry_price,
                    'Exit Price': exit_price,
                    'Return': rets
                })

        return pd.DataFrame(trades)

    except KeyError as e:
        print(f"Skipping {ticker}: missing column {e}")
        return pd.DataFrame()  # Return empty DataFrame on failure

    except Exception as e:
        print(f"Error with {ticker}: {e}")
        return pd.DataFrame()



In [ ]:
### Function to combine all trade data in one df
def simulate_all_trades(tickers, df, holding_days):
    """
    Simulate trades for each ticker and combine into one DataFrame.
    Skips tickers with missing price or signal columns.
    """
    all_trades = []

    for ticker in tickers:
        try:
            # Function call to run the trade sim
            trades_df = trade_sim(ticker, df, holding_days)
            # If the ticker never fell by 5% then it won't be included, so we omit it
            if not trades_df.empty:
                all_trades.append(trades_df)
        except Exception as e:
            print(f"Skipping {ticker}: {e}")
    
    if all_trades:
        return pd.concat(all_trades, ignore_index=True)
    else:
        return pd.DataFrame(columns = ['Ticker', 'Entry Date', 'Exit Date', 'Entry Price', 'Exit Price', 'Return'])
    
### Function to Simulate all Trades and store in one df
full_trades_df = simulate_all_trades(tickers, stock_data, holding_days = 14)
print(full_trades_df)


   Ticker  Entry Date  Exit Date  Entry Price  Exit Price    Return
0     MMM          64         77   184.841782  186.059998  0.006591
1     MMM          66         79   178.193222  193.509995  0.085956
2     AOS          64         77   124.284294  128.850006  0.036736
3     ABT          34         47   362.114349  315.505493 -0.128713
4     ABT          53         66   303.813477  298.459991 -0.017621
5     ABT          64         77   283.315094  293.390015  0.035561
6    ABBV          48         61   394.739990  385.779999 -0.022698
7     ACN          16         29   114.169998  113.099998 -0.009372
8     ACN          23         36   110.160004  104.739998 -0.049201
9     ACN          63         76    85.760002   94.470001  0.101562
10    ACN          64         77    83.639999   96.650002  0.155548
11    ACN          66         79    96.839996   96.059998 -0.008055
12    ACN          68         81    93.400002   96.650002  0.034797
13    ACN          72         85    87.500000  1

In [ ]:
### Evaluate Total Performance
def performance_metrics(df, holding_days):
    # Total Return
    total_rets = (1 + df['Return']).prod() - 1
    # Avg Return
    avg_rets = df['Return'].mean()
    # Win Rate = Num Winning Trades / Total Trades
    win_rate = (df['Return'] > 0).mean()
    # Sharpe Ratio
    sharpe = (avg_rets / df['Return'].std()) * np.sqrt(252 / holding_days)

    print(f"Number of Trades: {len(df)}")
    print(f"Total Return: {total_rets:.2%}")
    print(f"Average Trade Return: {avg_rets:.2%}")
    print(f"Win Rate: {win_rate:.2%}")
    print(f"Sharpe Ratio: {sharpe:.2f}")

### Function Return to Evaluate Total Performance
metrics = performance_metrics(full_trades_df, holding_days = 14)


Number of Trades: 16
Total Return: 61.99%
Average Trade Return: 3.32%
Win Rate: 62.50%
Sharpe Ratio: 1.89
